In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 23 10:49:48 2018

@author: hasee
"""
from nltk.corpus import brown
import nltk
from nltk.corpus import wordnet as wn
documents = nltk.corpus.brown.paras()
file = open( "combined.tab")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def loweCase_lemmatize(document):#pre-process the Brown corpus
    lemmaed_document = []
    for paragraph in document:
        paragraphs = []
        for sentence in paragraph:
            for word in sentence:
                word = lemmatizer.lemmatize(word.lower())
                paragraphs.append(word)
        lemmaed_document.append(set(paragraphs))
    return lemmaed_document

def count(word,document):# count the frequnency
    count = 0
    for paragraph in document:
            if word in paragraph:
                count += 1
    return count


tokenized_document = loweCase_lemmatize(documents)

def first_filter(document):# first_filter 
    dictionary = {}
    for aRow in file:
        list =  aRow.split('\t')
        if count(list[0],document) > 9 and count(list[1],document) > 9:
            tuple = (list[0],list[1])
            dictionary.setdefault(tuple,list[2])
    return dictionary

def synset_lemma(word):#find the primary sense
    lemmas = {}
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            if lemma.name() == word:
                lemmas.setdefault((synset,lemma),lemma.count())
    sorted_lemmas = sorted(lemmas.items(), lambda x, y: cmp(x[1], y[1]), reverse=True)
    if len(sorted_lemmas) == 1 and sorted_lemmas[0][0][0].pos() == "n":
        return True,sorted_lemmas[0][0][0]
    elif sorted_lemmas[0][1] > 5 and sorted_lemmas[0][1] >= 5*sorted_lemmas[1][1] and sorted_lemmas[0][0][0].pos() == "n":
        return True,sorted_lemmas[0][0][0] 
    else:
        return False,{}


def second_filter(document):#second_filter
    new_document = document
    keys = new_document.keys()
    for key in keys:
        if synset_lemma(key[0])[0] == False or synset_lemma(key[1])[0] == False:
         del new_document[key]
    return new_document

def get_primary_senes(pairs):# get the dictionary of word pairs and their primary senses
    new_pairs = {}
    for key in pairs:
        value = [synset_lemma(key[0])[1],synset_lemma(key[1])[1]]
        new_pairs.setdefault(key,value)
    return new_pairs



pairs = second_filter(first_filter(tokenized_document))
pairs1 = {}
for key in pairs:
    pairs1.setdefault(key,float((pairs[key].replace("\n",""))))
    

print "the total number is pairs is"
print len(pairs1)
print pairs1

pairs_with_primary_senes = get_primary_senes(pairs)


the total number is pairs is
24
{('journey', 'car'): 5.85, ('brother', 'monk'): 6.27, ('equipment', 'maker'): 5.91, ('coast', 'hill'): 4.38, ('baby', 'mother'): 7.85, ('canyon', 'landscape'): 7.53, ('doctor', 'personnel'): 5.0, ('soap', 'opera'): 7.94, ('monk', 'slave'): 0.92, ('psychology', 'doctor'): 6.42, ('car', 'automobile'): 8.94, ('type', 'kind'): 8.97, ('psychology', 'health'): 7.23, ('computer', 'laboratory'): 6.78, ('century', 'year'): 7.59, ('coast', 'forest'): 3.15, ('psychology', 'mind'): 7.69, ('school', 'center'): 3.44, ('journey', 'voyage'): 9.29, ('stock', 'egg'): 1.81, ('word', 'similarity'): 4.75, ('professor', 'doctor'): 6.62, ('planet', 'people'): 5.75, ('coast', 'shore'): 9.1}


In [2]:
def get_wu_scores(pairs):# get Wu-Palmer scores
    pairs_socre = {}
    for key in pairs:
        value = pairs[key]
        synset1 = value[0]
        synset2 = value[1]
        score = wn.wup_similarity(synset1, synset2)
        pairs_socre.setdefault(key,score)
    return pairs_socre
wu_pairs = get_wu_scores(pairs_with_primary_senes)
print wu_pairs

{('journey', 'car'): 0.09523809523809523, ('brother', 'monk'): 0.5714285714285714, ('equipment', 'maker'): 0.5, ('coast', 'hill'): 0.6666666666666666, ('baby', 'mother'): 0.5, ('canyon', 'landscape'): 0.3333333333333333, ('doctor', 'personnel'): 0.13333333333333333, ('soap', 'opera'): 0.2222222222222222, ('monk', 'slave'): 0.6666666666666666, ('psychology', 'doctor'): 0.1111111111111111, ('car', 'automobile'): 1.0, ('type', 'kind'): 0.9473684210526315, ('psychology', 'health'): 0.21052631578947367, ('computer', 'laboratory'): 0.35294117647058826, ('century', 'year'): 0.8333333333333334, ('coast', 'forest'): 0.16666666666666666, ('psychology', 'mind'): 0.5714285714285714, ('school', 'center'): 0.13333333333333333, ('journey', 'voyage'): 0.8571428571428571, ('stock', 'egg'): 0.11764705882352941, ('word', 'similarity'): 0.3333333333333333, ('professor', 'doctor'): 0.5, ('coast', 'shore'): 0.9090909090909091, ('planet', 'people'): 0.18181818181818182}


In [3]:
import math
def get_PPMI(word1,word2,document):# get the ppmi of 2 words
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_count = 0.0
    for paragraph in document:
        total_count += len(paragraph)
        if word1 in paragraph:
            word1_count += 1
            if word2 in paragraph:
                both_count += 1
        elif word2 in paragraph:
            word2_count += 1
    if both_count == 0:
        return 0
    else:
        pmi = math.log((both_count/total_count)/((word1_count/total_count)*(word2_count/total_count)), 2)
        return max(pmi,0)


def get_PPMI_pair(pair,document):#get the dictionary of words pairs and their PPMI
    new_pairs = {}
    for key in pairs:
        value = get_PPMI(key[0],key[1],document)
        new_pairs.setdefault(key,value)
    return new_pairs
PPMI_pairs = get_PPMI_pair(pairs,tokenized_document)
print PPMI_pairs

{('journey', 'car'): 0, ('brother', 'monk'): 8.588453524677158, ('equipment', 'maker'): 10.19395451157477, ('coast', 'hill'): 6.8415427432365234, ('baby', 'mother'): 8.776192449102656, ('canyon', 'landscape'): 0, ('doctor', 'personnel'): 7.8782012138153705, ('soap', 'opera'): 9.872906914145858, ('monk', 'slave'): 0, ('psychology', 'doctor'): 9.19197535489398, ('car', 'automobile'): 9.178414446597797, ('type', 'kind'): 6.301105661499941, ('psychology', 'health'): 0, ('computer', 'laboratory'): 0, ('century', 'year'): 6.506664771776898, ('coast', 'forest'): 8.713787196038151, ('psychology', 'mind'): 8.403103611436123, ('school', 'center'): 6.4280926584302005, ('journey', 'voyage'): 0, ('stock', 'egg'): 7.474479027764361, ('word', 'similarity'): 0, ('professor', 'doctor'): 0, ('planet', 'people'): 6.028474169984525, ('coast', 'shore'): 10.394796384230341}


In [ ]:
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.spatial.distance import cosine as cos_distance
from sklearn.feature_extraction.text import TfidfTransformer
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW

texts = []
for paragraph in tokenized_document:
    texts.append(get_BOW(paragraph))   
    
vectorizer = DictVectorizer()
matrix = vectorizer.fit_transform(texts)
matrix = np.transpose(matrix)
transformer = TfidfTransformer(smooth_idf=False,norm=None)
matrix = transformer.fit_transform(matrix)
svd = TruncatedSVD(n_components=500)
matrix = svd.fit_transform(matrix)

def get_row(word):
    return vectorizer.get_feature_names().index(word)

def svd_similarity_pairs(pairs,matrix):
    new_pairs = {}
    for key in pairs:
        vectoe1  = matrix[get_row(key[0])]
        vectoe2  = matrix[get_row(key[1])]
        value = 1 - cos_distance(vectoe1,vectoe2)
        new_pairs.setdefault(key,value)
    return new_pairs
    
svd_pairs = svd_similarity_pairs(pairs,matrix)
print svd_pairs

{('journey', 'car'): -0.015060280564988648, ('brother', 'monk'): 0.07901775098870045, ('equipment', 'maker'): 0.2885449049623524, ('coast', 'hill'): 0.1841347061981673, ('baby', 'mother'): 0.29760477229827775, ('canyon', 'landscape'): 0.09013870783128552, ('doctor', 'personnel'): 0.05418676478243645, ('soap', 'opera'): 0.014608558896880952, ('monk', 'slave'): -0.01587697235481622, ('psychology', 'doctor'): 0.06400045349324868, ('car', 'automobile'): 0.32473844604925794, ('type', 'kind'): 0.023739016763977716, ('psychology', 'health'): 0.04362310736055919, ('computer', 'laboratory'): 0.12677766307459515, ('century', 'year'): 0.06258368801916647, ('coast', 'forest'): 0.11656406706199751, ('psychology', 'mind'): 0.11774001276758295, ('school', 'center'): 0.04220126640079802, ('journey', 'voyage'): 0.12643390494882167, ('stock', 'egg'): 0.09995085202884646, ('word', 'similarity'): 0.0030614516447731166, ('professor', 'doctor'): 0.05309113583106195, ('planet', 'people'): 0.02550641322033042

In [ ]:
import gensim
sentences = nltk.corpus.brown.sents()
def train(text):
    model = gensim.models.Word2Vec(text,size = 500,iter = 50)
    return model

model = train(sentences)
    
def get_Word2vec_pair(pair,model):
    new_pairs = {}
    for key in pairs:
        value = model.wv.similarity(key[0],key[1])
        new_pairs.setdefault(key,value)
    return new_pairs

word2vec_pairs = get_Word2vec_pair(pairs,model)
print word2vec_pairs

G:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
from scipy import stats
def dic_to_list(dic):
    return dic.items()

dataset = [word2vec_pairs,svd_pairs,PPMI_pairs,wu_pairs]
setname = ["word2vec_pairs","svd_pairs","PPMI_pairs","wu_pairs"]



def co_efficient(dataset,setname):
    for i in range(len(dataset)):     
            set1 = dataset[i].values()
            pearsonr = stats.pearsonr(set1,pairs1.values())
            print "pearsonr for set "+ setname[i]+" and gold standardis " + str(pearsonr)
            
print co_efficient(dataset,setname)



